In [13]:
import requests
import pandas as pd
from datetime import date
import numpy as np

# URL template (corrected)
url = "https://kite.zerodha.com/oms/instruments/historical/256265/5minute?user_id=HHS112&oi=1&from=2025-06-11&to=2025-06-15"

# Instrument tokens
tickerData = {"NIFTY": {"id": 256265}, "BANKNIFTY": {"id": 260105}}

# Interval mapping
KI = {
    "3m": "3minute",
    "5m": "5minute",
    "15m": "15minute",
    "60m": "60minute",
    "day": "day",
}

# Function to get candle data
def getCandles(symbol, fromDate, toDate, timeframe):
    headers = {
        "authorization": "enctoken l9RBs2mUaCOtHh76/Al766xp7yJ1ndunq9zE2PslZvpMEXu0ccRXlWhWHNUOyJX19msbiNnAcd37PdO0yb1ikRiT3q1N7v2hutAK2FB3s2GAGxkfVgusyQ=="
        # Replace this with your actual token securely
    }
    curUrl = url.format(tickerData[symbol]["id"], timeframe, fromDate, toDate)
    print("Requesting:", curUrl)

    try:
        session = requests.session()
        response = session.get(curUrl, headers=headers)
        r = response.json()
    except Exception as error:
        print("API request failed:", error)
        return pd.DataFrame()

    try:
        history = r["data"]["candles"]
        history_df = pd.DataFrame(history)
        history_df = history_df.rename(
            columns={
                0: "Datetime",
                1: "open",
                2: "high",
                3: "low",
                4: "close",
                5: "volume",
            }
        )
        history_df["Datetime"] = pd.to_datetime(history_df["Datetime"])
        history_df.set_index("Datetime", inplace=True)
        return history_df
    except Exception as error:
        print("Data processing failed:", error)
        return pd.DataFrame()

def ATR(df, n):
    df = df.copy()
    df["H-L"] = abs(df["high"] - df["low"])
    df["H-PC"] = abs(df["high"] - df["close"].shift(1))
    df["L-PC"] = abs(df["low"] - df["close"].shift(1))
    df["TR"] = df[["H-L", "H-PC", "L-PC"]].max(axis=1, skipna=False)
    df["ATR"] = df["TR"].ewm(span=n, adjust=False, min_periods=n).mean()
    df = df.drop(["H-L", "H-PC", "L-PC", "TR"], axis=1)
    return df


def ADX(df_dict, n=14):
    df = df_dict.copy()
    df = ATR(df, n)  # Proper usage

    df["upmove"] = df["high"] - df["high"].shift(1)
    df["downmove"] = df["low"].shift(1) - df["low"]

    df["+dm"] = np.where(
        (df["upmove"] > df["downmove"]) & (df["upmove"] > 0), df["upmove"], 0
    )
    df["-dm"] = np.where(
        (df["downmove"] > df["upmove"]) & (df["downmove"] > 0), df["downmove"], 0
    )

    plus_dm_ewm = df["+dm"].ewm(alpha=1 / n, min_periods=n).mean()
    minus_dm_ewm = df["-dm"].ewm(alpha=1 / n, min_periods=n).mean()

    df["+di"] = 100 * (plus_dm_ewm / df["ATR"])
    df["-di"] = 100 * (minus_dm_ewm / df["ATR"])

    dx = 100 * abs(df["+di"] - df["-di"]) / (df["+di"] + df["-di"]).replace(0, np.nan)
    df["ADX"] = dx.ewm(alpha=1 / n, min_periods=n).mean()

    return df



# Example usage
print("Start")
symbol_data = getCandles("NIFTY", "2025-06-11", "2025-06-15", KI["5m"])
if not symbol_data.empty:
    adx_data = ADX(symbol_data)
    print(adx_data[["+di", "-di", "ADX"]].tail(10))
else:
    print("No data returned.")


Start
Requesting: https://kite.zerodha.com/oms/instruments/historical/256265/5minute?user_id=HHS112&oi=1&from=2025-06-11&to=2025-06-15
                                 +di        -di        ADX
Datetime                                                  
2025-06-13 14:40:00+05:30  19.987189  24.442339  15.464314
2025-06-13 14:45:00+05:30  17.483667  25.310321  15.666088
2025-06-13 14:50:00+05:30  16.612333  28.793180  16.463282
2025-06-13 14:55:00+05:30  14.486532  31.042519  17.884731
2025-06-13 15:00:00+05:30  20.709542  24.789744  17.247794
2025-06-13 15:05:00+05:30  27.104227  22.301682  16.710137
2025-06-13 15:10:00+05:30  27.067881  22.110906  16.236520
2025-06-13 15:15:00+05:30  25.809855  23.802751  15.365736
2025-06-13 15:20:00+05:30  27.429110  22.780540  14.929492
2025-06-13 15:25:00+05:30  30.440727  21.529145  15.087928
